In [2]:
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 2.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 KB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.3 MB/s eta 0:00:0000:0100:01


In [96]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [97]:
threshold_sp = 20     # Salt-and-pepper noise threshold

In [98]:
detected_noises = []  # Detected noise types


In [99]:
def detect_salt_pepper_noise(image):

    # Check for salt-and-pepper noise
    median_diff = np.mean(np.abs(image - cv2.medianBlur(image, 3)))
    if median_diff > threshold_sp:
        detected_noises.append('salt_pepper')

In [111]:
def detect_sine_wave(image):
   
    
    # Perform the 2D FFT of the image
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)  # Shift the zero frequency component to the center
    
    # Calculate the magnitude spectrum for visualization (not necessary for detection)
    magnitude_spectrum = np.log(np.abs(fshift) + 1)  # +1 to avoid log(0)

    # Define the center of the frequency spectrum
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2

    # Optionally filter out low frequencies (central part of spectrum)
    fshift[crow - 30:crow + 30, ccol - 30:ccol + 30] = 0

    # Inverse FFT to reconstruct the image without low-frequency components
    f_ishift = np.fft.ifftshift(fshift)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)

    # Compute the difference between the original image and the processed image
    sine_wave_diff = np.mean(np.abs(image - img_back))

    # Adjust threshold based on empirical observations
    threshold = 10  # A higher threshold for sine wave detection

    # If the difference is greater than the threshold, consider it as sine wave noise
    if sine_wave_diff > threshold:
        detected_noises.append('sine_wave')


In [101]:
def detect_image_is_blurred(image):
    # Check for blurred image
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    laplacian_var = np.var(laplacian)
    if laplacian_var < 10:
        detected_noises.append('blurred')

In [102]:
def detect_low_light(image):
    # Check for low light noise
    low_light_diff = np.mean(image)
    if low_light_diff < 50:
        detected_noises.append('low_light')

In [103]:
def detect_noises(image, detect_noises):
    detect_salt_pepper_noise(image)
    detect_sine_wave(image)
    detect_low_light(image)
    detect_image_is_blurred(image)


In [104]:
def preprocess_image(image, noise_types):

    if noise_types is None:
        return image

    processed_image = image.copy()


    if 'salt_pepper' in noise_types:

        processed_image = cv2.medianBlur(cv2.blur(processed_image, (1, 9)), 5)  # Apply a median filter first
        _, processed_image = cv2.threshold(processed_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # Apply Otsu's thresholding
    


    return processed_image

In [112]:
# Read barcode image
image = cv2.imread('Test Case Images/07 - mal7 w felfel.jpg', cv2.IMREAD_GRAYSCALE)

In [113]:
detect_noises(image, detected_noises)
print(f"Detected noise types: {detected_noises}")

Detected noise types: ['sine_wave', 'salt_pepper', 'sine_wave', 'salt_pepper', 'sine_wave']


In [11]:
# Apply preprocessing
processed_image = preprocess_image(image, detected_noises)

In [12]:
#Save or display the result
cv2.imwrite('processed_barcode.jpg', processed_image)

True

In [ ]:


def show_image(title, image):
    plt.figure(figsize=(10, 10))
    plt.title(title)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

def crop_barcode(image_path, output_path):
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to read image from {image_path}")
        return

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise and improve edge detection
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)

    # Apply edge detection
    edged = cv2.Canny(blurred, 50, 200, 255)

    # Apply morphological operations to close gaps in the edges
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
    closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)

    # Show the edged image
    show_image("Edged Image", edged)
    show_image("Closed Image", closed)

    # Find contours in the closed image
    contours, _ = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours by area, keeping only the largest one
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    barcode_contour = None

    # Loop over the contours to find the one that likely represents the barcode
    for contour in contours:
        # Approximate the contour
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)

        # If the approximated contour has four points, we can assume we have found the barcode
        if len(approx) == 4:
            barcode_contour = approx
            break

    if barcode_contour is not None:
        # Draw the contour on the image
        cv2.drawContours(image, [barcode_contour], -1, (0, 255, 0), 2)
        show_image("Detected Barcode", image)

        # Get the bounding box of the barcode contour
        x, y, w, h = cv2.boundingRect(barcode_contour)

        # Crop the image to the bounding box
        cropped_image = image[y:y+h, x:x+w]

        # Save the cropped image
        cv2.imwrite(output_path, cropped_image)
        print(f"Cropped image saved to {output_path}")
    else:
        print("No barcode detected")


In [ ]:
# Example usage
crop_barcode('07 - mal7 w felfel.jpg', 'cropped_image.jpg')